<img src="../fasp/runner/credits/images/FullHouse.png" style="float: right;">

### Submit Plenary GWAS workflow using DRS ids from both Anvil and BioDataCatalyst DRS

Job is submitted to DNAStack WES server. 

Submission is successful, but jobs fail.

Current issue is with how the signed URLs are handled on the WES server.

In [1]:
#  IMPORTS
#import sys
import datetime

from fasp.runner import FASPRunner

# The implementations we're using
from fasp.search import DataConnectClient
from fasp.loc import DRSMetaResolver
from fasp.workflow import DNAStackWESClient


faspRunner = FASPRunner()

# Step 1 - Discovery
# query for relevant DRS objects
searchClient = DataConnectClient('https://data.publisher.dnastack.com/data-connect/', debug=False)

query = '''
SELECT file_name, compact_drs_id, hostbased_drs_id, drs_id 
FROM collections.public_datasets.onek_genomes_onek_recal_variants_drs 
WHERE chromosome = 'chr21'
    AND annotated = false
'''

query_job = searchClient.runQuery(query)  # Send the query

drsResolver = DRSMetaResolver(getReg=False)	

wesClient = DNAStackWESClient('~/.keys/dnastack_wes_credentials.json')

# repeat steps 2 and 3 for each row of the query
# this example should find id's for the same file in both BioDataCatalyst and Anvil
for row in query_job:
    drs_id = row[1]
    print("vcffile={}, compact drsID={}".format(row[0], drs_id))

    # Step 2 - Use DRS to get the URL
    vcfurl = drsResolver.getAccessURL(drs_id, 'gs')
    drsClient, localid = drsResolver.getClient(drs_id) 
    fileSize = drsResolver.getObject(drs_id)['size']
    
    # Step 3 - Run a pipeline on the file at the drs url
    pipeline_id = wesClient.runGWASWorkflow(vcfurl, 'gs://dnastack-public-bucket/thousand_genomes_meta.csv')
    print('submitted:{}'.format(pipeline_id))

    outfile = ''
    via = 'WES'
    note = 'GWAS'

    time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    faspRunner.logRun(time, via, note,  pipeline_id, outfile, str(fileSize),
        searchClient, drsClient, wesClient)

Running FASPNotebookGWAS.ipynb
_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
vcffile=CCDG_13607_B01_GRM_WGS_2019-02-19_chr21.recalibrated_variants.vcf.gz, compact drsID=bdc:dg.4503/dbd55e76-1100-40b3-b420-0eaeee478fbc
submitted:387989d1-7cae-45a0-82ce-e15d53353ba3
vcffile=CCDG_13607_B01_GRM_WGS_2019-02-19_chr21.recalibrated_variants.vcf.gz, compact drsID=anv:dg.ANV0/737247da-f5da-49a7-86ec-737978eb8293
submitted:0c089b18-aa8c-4007-b131-485527a6a0eb
